## CMSC 35300 Final Project

In [1]:
import time
import numpy as np
from scipy.sparse import csc_matrix
from pandas import read_csv

In [2]:
k = 10
lambd = 4
N = 5

In [3]:
df = read_csv('ml-25m/ratings.csv')
df = df.drop('timestamp', axis=1)
df.loc[:,'rating'] *= 2
df = df.astype(int)

In [4]:
df.head(3)

,userId,movieId,rating
0,1,296,10
1,1,306,7
2,1,307,10


In [5]:
max_userId = df.max().loc['userId']
max_movieId = df.max().loc['movieId']
observation_number = df.index[-1]+1
slices = observation_number//N

In [6]:
print(max_movieId)
print(max_userId)
print(observation_number)
print(slices)

209171
162541
25000095
5000019


In [7]:
observed_data = df.to_numpy()
observed_data[:,:-1] = observed_data[:,:-1] -1
observed_data.shape

(25000095, 3)

In [8]:
observed_data[:3,:]

array([[  0, 295,  10],
       [  0, 305,   7],
       [  0, 306,  10]])

In [9]:
df.head(3)

,userId,movieId,rating
0,0,295,10
1,0,305,7
2,0,306,10


In [10]:
split_sets = np.zeros((N,slices, 3), dtype=int)

In [11]:
random_indices = np.arange(observation_number)
np.random.shuffle(random_indices)
random_indices = random_indices.reshape((N,slices))

In [12]:
for i in range(N):
    for j in range(slices):
        split_sets[i,j,:] = observed_data[random_indices[i,j],:]

In [14]:
print(observed_data[random_indices[0,0],:])
print(split_sets[0,0,:])
print(random_indices[0,0])
df.loc[(df['userId'] == split_sets[0,0,0]) & (df['movieId'] == split_sets[0,0,1])]

[42220 55954     6]
[42220 55954     6]
6514459


,userId,movieId,rating
6514459,42220,55954,6


In [26]:
def power_factorization(n, X0, Y0, t_set):

    X = X0
    Y = Y0
    X1 = np.zeros((k,max_userId))
    Y1 = np.zeros((k,max_movieId))
    
    inner_startTime = time.time()

    inner_iteration = 0
    print('Current inner-iteration: ' + str(inner_iteration))
    print('np.linalg.norm(X1-X) = ' + str(np.linalg.norm(X1-X)))
    print('np.linalg.norm(Y1-Y) = ' + str(np.linalg.norm(Y1-Y))+'\n')

    Xnorm_diff = np.linalg.norm(X1-X)
    Ynorm_diff = np.linalg.norm(Y1-Y)

    while True:
        inner_iteration += 1
        print('Current inner-iteration: ' + str(inner_iteration))


        u_to_sums = {}
        for s in range(slices):
            u, i, r = t_set[s,:]
            if u not in u_to_sums.keys():
                u_to_sums[u] = {'sum_yi_yiT':np.zeros((k,k)), 'sum_rui_yi':np.zeros((k,1))}
            u_to_sums[u]['sum_yi_yiT'] += Y[:,i:i+1]@Y[:,i:i+1].T
            u_to_sums[u]['sum_rui_yi'] += r*Y[:,i:i+1]
        for u in u_to_sums.keys():
            X1[:,u:u+1] = np.linalg.inv(u_to_sums[u]['sum_yi_yiT']+lambd*np.diag(np.ones(k)))@u_to_sums[u]['sum_rui_yi']

        executionTime = (time.time() - inner_startTime)
        print('Elapsed time in seconds: ' + str(executionTime))

        Xnorm_diff = np.linalg.norm(X1-X)
        print('np.linalg.norm(X1-X) = ' + str(Xnorm_diff)+'\n')

        if Xnorm_diff<1e-5 and Ynorm_diff<1e-5:
            break

        X = X1


        i_to_sums = {}
        for s in range(slices): # i starts from 0
            u, i, r = t_set[s,:]
            if i not in i_to_sums.keys():
                i_to_sums[i] = {'sum_xu_xuT': np.zeros((k,k)), 'sum_rui_xu': np.zeros((k,1))}
            i_to_sums[i]['sum_xu_xuT'] += X[:,u:u+1]@X[:,u:u+1].T
            i_to_sums[i]['sum_rui_xu'] += r*X[:,u:u+1]
        for i in i_to_sums.keys():
            Y1[:,i:i+1] = np.linalg.inv(i_to_sums[i]['sum_xu_xuT']+lambd*np.diag(np.ones(k)))@i_to_sums[i]['sum_rui_xu']

        executionTime = (time.time() - inner_startTime)
        print('Elapsed time in seconds: ' + str(executionTime))

        Ynorm_diff = np.linalg.norm(Y1-Y)
        print('np.linalg.norm(Y1-Y) = ' + str(Ynorm_diff)+'\n')

        if Xnorm_diff<1e-5 and Ynorm_diff<1e-5:
            break

        Y = Y1


    executionTime = (time.time() - inner_startTime)
    print('Execution time in seconds: ' + str(executionTime))
    print('Total inner-iterations: ' + str(inner_iteration))
    
    return X, Y

In [17]:
X0 = np.random.uniform(0,10,(k, max_userId))
Y0 = np.random.uniform(0,10,(k, max_movieId))

In [19]:
for n in range(N):
    training_set = np.zeros(((N-1)*slices,3), dtype=int)
    test_set = np.zeros((slices,3), dtype=int)
    temp = np.append(split_sets[:n,:,:], split_sets[n+1:,:,:],axis=0)
    for t in range(N-1):
        training_set[t*slices:(t+1)*slices,:] = temp[t:t+1,:,:]
    test_set[:,:] = split_sets[n:n+1,:,:]

    X, Y = power_factorization(n, X0, Y0, training_set)

    

In [ ]:
np.savetxt('X.csv', X, delimiter=',')
np.savetxt('Y.csv', Y, delimiter=',')

In [ ]:
# X = np.loadtxt('X.csv', delimiter=',')
# Y = np.loadtxt('Y.csv', delimiter=',')

In [ ]:
sum_first_term = 0
for userId, movieId in ratings.keys():
    sum_first_term += (ratings[(userId,movieId)] - X[:,userId].T@Y[:,movieId])**2
print("sum_first_term = " + str(sum_first_term))
print("sum_first_term + lambda*Frobenius_norm = " + str(sum_first_term + lambd*(np.linalg.norm(X)+np.linalg.norm(Y))))

In [ ]:
np.sqrt(sum_first_term/df.index[-1])